# Brain MNIST Classification
In this data set we will be using the MindBigData dataset. I will be solely using just the MindWave file. This dataset contains EEG data when a person is shown a digit (0-9). The goal is to classify based on EEG data what digit was shown to the person.

In [64]:
#importing dataset
import pandas as pd

data=pd.read_csv("MW.txt", sep='\t', header=None, names=["id", "event", "channel", "num", "size", "eeg"])

#plotting first 5 samples
data.head()

,id,event,channel,num,size,eeg
0,0,MW,FP1,0,1017,"38,48,51,44,48,56,56,41,20,-3,-9,-1,6,5,6,17,2..."
1,1,MW,FP1,1,889,"83,74,65,65,66,55,43,25,18,20,26,33,41,38,28,3..."
2,2,MW,FP1,4,1017,"19,10,-2,-9,-5,3,8,7,8,2,-10,-21,-22,-27,-30,-..."
3,3,MW,FP1,1,1017,"17,29,36,26,21,17,17,13,17,21,25,26,20,5,-5,-1..."
4,4,MW,FP1,8,952,"77,74,69,70,76,75,76,75,67,66,76,93,98,82,55,3..."


In [65]:
#preprocessing the eeg data
import numpy as np
import matplotlib.pyplot as plt



0        38,48,51,44,48,56,56,41,20,-3,-9,-1,6,5,6,17,2...
1        83,74,65,65,66,55,43,25,18,20,26,33,41,38,28,3...
2        19,10,-2,-9,-5,3,8,7,8,2,-10,-21,-22,-27,-30,-...
3        17,29,36,26,21,17,17,13,17,21,25,26,20,5,-5,-1...
4        77,74,69,70,76,75,76,75,67,66,76,93,98,82,55,3...
                               ...                        
67630    138,135,123,96,59,38,11,16,44,67,70,73,67,33,4...
67631    4,5,-7,-12,-11,-17,2,52,73,55,12,-22,-7,38,43,...
67632    96,163,208,179,129,114,140,155,108,56,28,9,-8,...
67633    -30,4,53,85,84,72,80,85,65,23,-26,-42,-56,-146...
67634    41,27,41,76,88,80,83,76,60,72,112,124,100,60,3...
Name: eeg, Length: 67635, dtype: object